In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub
import random
import math
import pandas as pd
from IPython.display import display, Image, clear_output

# Load the model
model = hub.load("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow1/openimages-v4-ssd-mobilenet-v2/1").signatures["default"]

# Define the target classes to detect
target_classes = []

# Initialize a dictionary to store color codes
colorcodes = {}

# Initialize a list to store detection data
detections_data = []

# Function to draw bounding boxes
def drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color):
    im_height, im_width, _ = image.shape
    left, top, right, bottom = int(xmin * im_width), int(ymin * im_height), int(xmax * im_width), int(ymax * im_height)
    cv2.rectangle(image, (left, top), (right, bottom), color=color, thickness=2)

    FONT_SCALE = 0.5
    THICKNESS_SCALE = 1
    TEXT_Y_OFFSET_SCALE = 0.01

    cv2.rectangle(
        image,
        (left, top - int(20)),
        (right, top),
        color=color,
        thickness=-1
    )

    cv2.putText(
        image,
        namewithscore,
        (left, top - int(5)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=FONT_SCALE,
        thickness=THICKNESS_SCALE,
        color=(255, 255, 255)
    )

# Function to process detections and draw them on the image
def draw(image, boxes, classnames, scores):
    global detections_data
    boxesidx = tf.image.non_max_suppression(boxes, scores, max_output_size=10, iou_threshold=0.4, score_threshold=0.3)

    for i in boxesidx:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")

        if classname in target_classes:
            if classname in colorcodes.keys():
                color = colorcodes[classname]
            else:
                c1 = random.randrange(0, 255, 30)
                c2 = random.randrange(0, 255, 25)
                c3 = random.randrange(0, 255, 50)
                colorcodes.update({classname: (c1, c2, c3)})
                color = colorcodes[classname]

            namewithscore = "{}:{}".format(classname, int(100 * scores[i]))
            drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color)

            # Store detection data
            detections_data.append({
                "Class": classname,
                "Confidence Score": scores[i],
                "ymin": ymin,
                "xmin": xmin,
                "ymax": ymax,
                "xmax": xmax
            })

    return image

# Real-time object detection
def detect_objects_in_real_time():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width = 400, 400
        frame = cv2.resize(frame, (width, height))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        converted_img = tf.image.convert_image_dtype(image_rgb, tf.float32)[tf.newaxis, ...]

        detection = model(converted_img)
        result = {key: value.numpy() for key, value in detection.items()}

        image_with_boxes = draw(frame, result['detection_boxes'], result['detection_class_entities'], result["detection_scores"])

        clear_output(wait=True)
        _, encoded_image = cv2.imencode('.jpg', image_with_boxes)
        display(Image(data=encoded_image))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()

    # Save detections to Excel file
    df = pd.DataFrame(detections_data)
    df.to_excel("detections.xlsx", index=False)
    print("Detections saved to detections.xlsx")

# Run the real-time object detection
detect_objects_in_real_time()


In [5]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub
import random
import math
import pandas as pd
from datetime import datetime
import time  # Import time module

# Load the model
model = hub.load("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow1/openimages-v4-ssd-mobilenet-v2/1").signatures["default"]

# Initialize a dictionary to store color codes
colorcodes = {}

# Function to draw bounding boxes
def drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color):
    im_height, im_width, _ = image.shape
    left, top, right, bottom = int(xmin * im_width), int(ymin * im_height), int(xmax * im_width), int(ymax * im_height)
    cv2.rectangle(image, (left, top), (right, bottom), color=color, thickness=2)

    FONT_SCALE = 0.5
    THICKNESS_SCALE = 1
    TEXT_Y_OFFSET_SCALE = 0.01

    cv2.rectangle(
        image,
        (left, top - int(20)),
        (right, top),
        color=color,
        thickness=-1
    )

    cv2.putText(
        image,
        namewithscore,
        (left, top - int(5)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=FONT_SCALE,
        thickness=THICKNESS_SCALE,
        color=(255, 255, 255)
    )

# Function to process detections and draw them on the image
def draw(image, boxes, classnames, scores):
    # Filter detections based on scores
    boxesidx = tf.image.non_max_suppression(boxes, scores, max_output_size=10, iou_threshold=0.4, score_threshold=0.3)
    detections = []

    for i in boxesidx:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        score = scores[i]
        detections.append((classname, score))

        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0, 255, 30)
            c2 = random.randrange(0, 255, 25)
            c3 = random.randrange(0, 255, 50)
            colorcodes.update({classname: (c1, c2, c3)})
            color = colorcodes[classname]

        namewithscore = "{}:{}".format(classname, int(100 * score))
        drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color)

    return image, detections

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create an empty DataFrame to store detections
df = pd.DataFrame(columns=["Time", "Class", "Score"])

# Start the webcam feed
try:
    while True:
        start_time = time.time()  # Record the start time of the loop

        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame for consistent processing
        frame = cv2.resize(frame, (640, 480))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        converted_img = tf.image.convert_image_dtype(frame_rgb, tf.float32)[tf.newaxis, ...]

        # Perform detection
        detection = model(converted_img)
        result = {key: value.numpy() for key, value in detection.items()}

        # Draw bounding boxes and get detections
        frame_with_boxes, detections = draw(frame, result['detection_boxes'], result['detection_class_entities'], result["detection_scores"])

        # Display the frame with bounding boxes
        cv2.imshow('Real-time Object Detection', frame_with_boxes)

        # Save detections into DataFrame
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        if detections:
            new_rows = pd.DataFrame([(current_time, classname, score) for classname, score in detections], columns=["Time", "Class", "Score"])
            df = pd.concat([df, new_rows], ignore_index=True)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Ensure the loop runs only twice per second
        elapsed_time = time.time() - start_time
        if elapsed_time < 0.5:  # If processing took less than 0.5 seconds, wait for the remaining time
            time.sleep(0.5 - elapsed_time)

finally:
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()

    # Save the detections to an Excel file
    df.to_excel("detections.xlsx", index=False)
    print("Detections saved to detections.xlsx")


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\Users\Sami\AppData\Local\Temp\ipykernel_19176\4246426242.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_rows], ignore_index=True)


Detections saved to detections.xlsx
